In [ ]:
%pip install --quiet -U langgraph langsmith
%pip install --quiet -U "langchain[openai]"
%pip install requests

In [2]:
import os

# Set environment variables for API keys
os.environ["GOOGLE_MAPS_API_KEY"] = "example"
os.environ["OPENAI_API_KEY"] = "example"

In [3]:
import requests
from langchain_core.tools import tool

@tool
def tool_google_maps_search(text_query: str):
    """"Searches Google Maps for places based on a text query."""
    api_key = os.environ.get("GOOGLE_MAPS_API_KEY")
    if not api_key:
        raise ValueError("API key for Google Maps is not set in the environment variables.")

    url = "https://places.googleapis.com/v1/places:searchText"
    
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.id,places.attributions,places.displayName,places.formattedAddress,places.location,places.googleMapsLinks"
    }

    payload = {
        "textQuery": text_query,
    }

    response = requests.post(url, json=payload, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Error at Google Maps API: {response.text}")

    return response.json()

In [4]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [5]:
from typing import Annotated

from langchain.chat_models import init_chat_model
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# Define the tool that will be used in the chatbot
tools = [tool_google_maps_search]

# Initialize the chat model with the tools binded to it
llm = init_chat_model("openai:gpt-4.1")
llm_with_tools = llm.bind_tools(tools)

# Chatbot node that will handle the conversation
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Create the Chatbot node
graph_builder.add_node("chatbot", chatbot)

# Create the ToolNode that will handle the tools
tool_node = ToolNode(tools)
graph_builder.add_node("tools", tool_node)

# Define the edges of the graph
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile(checkpointer=memory)


In [ ]:
from IPython.display import Image, display

(Image(graph.get_graph().draw_mermaid_png()))

In [7]:
config = {"configurable": {"thread_id": "1"}}

In [ ]:
user_input = ""

events = graph.stream(
    {
        "messages": [
            {
                "role": "system",
                "content": (
                    "You are a location-aware assistant. "
                    "You receive user questions involving geographic locations and have access to a tool "
                    "that queries the Google Maps Places API. "
                    "Your job is to respond with relevant places, including their name, address, and direct Google Maps link."
                ),
            },
            {"role": "user", "content": user_input},
        ]
    },
    config,
    stream_mode="values",
)

for event in events:
    event["messages"][-1].pretty_print()